In [1]:
from line_solver import *

import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')
n = 5  # number of items
m = 2  # cache capacity

delay = Delay(model, 'Delay')
cacheNode = Cache(model, 'Cache', n, m, ReplacementStrategy.FIFO)
jobClass = ClosedClass(model, 'JobClass', 1, delay, 0)
hitClass = ClosedClass(model, 'HitClass', 0, delay, 0)
missClass = ClosedClass(model, 'MissClass', 0, delay, 0)

delay.set_service(jobClass, Exp(1))

# Uniform item references - create Matrix from numpy array
prob_array = np.ones(n) / n
pAccess = DiscreteSampler([1/n] * n)
cacheNode.set_read(jobClass, pAccess)

cacheNode.set_hit_class(jobClass, hitClass)
cacheNode.set_miss_class(jobClass, missClass)
P = model.init_routing_matrix()
P.set(jobClass, jobClass, delay, cacheNode, 1.0)
P.set(hitClass, jobClass, cacheNode, delay, 1.0)
P.set(missClass, jobClass, cacheNode, delay, 1.0)

model.link(P)

In [3]:
solver = np.empty(3, dtype=object)
AvgTable = np.empty(3, dtype=object)

In [4]:
solver[0] = CTMC(model, keep=False, seed=1)
AvgTable[0] = solver[0].avg_node_table()

                Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0              Delay   JobClass   1.0   1.0    1.0     1.0   1.0   1.0
3              Cache   JobClass   0.0   0.0    0.0     0.0   1.0   0.0
4              Cache   HitClass   0.0   0.0    0.0     0.0   0.0   0.4
5              Cache  MissClass   0.0   0.0    0.0     0.0   0.0   0.6
6  CS_Cache_to_Delay   JobClass   0.0   0.0    0.0     0.0   0.0   1.0
7  CS_Cache_to_Delay   HitClass   0.0   0.0    0.0     0.0   0.4   0.0
8  CS_Cache_to_Delay  MissClass   0.0   0.0    0.0     0.0   0.6   0.0


In [5]:
model.reset()

In [14]:
solver[1] = SSA(model, samples=10000, verbose=True, method='serial', seed=1)
AvgTable[1] = solver[1].avg_node_table()

SSA samples:   10000
                Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0              Delay   JobClass   1.0   1.0    1.0     1.0   1.0   1.0
3              Cache   JobClass   0.0   0.0    0.0     0.0   1.0   0.0
4              Cache   HitClass   0.0   0.0    0.0     0.0   0.0   0.4
5              Cache  MissClass   0.0   0.0    0.0     0.0   0.0   0.6
6  CS_Cache_to_Delay   JobClass   0.0   0.0    0.0     0.0   0.0   1.0
7  CS_Cache_to_Delay   HitClass   0.0   0.0    0.0     0.0   0.4   0.0
8  CS_Cache_to_Delay  MissClass   0.0   0.0    0.0     0.0   0.6   0.0


In [7]:
solver[2] = MVA(model, seed=1)
AvgTable[2] = solver[2].avg_node_table()

                Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0              Delay   JobClass   1.0   1.0    1.0     1.0   1.0   1.0
3              Cache   JobClass   0.0   0.0    0.0     0.0   1.0   0.0
4              Cache   HitClass   0.0   0.0    0.0     0.0   0.0   0.4
5              Cache  MissClass   0.0   0.0    0.0     0.0   0.0   0.6
6  CS_Cache_to_Delay   JobClass   0.0   0.0    0.0     0.0   0.0   1.0
7  CS_Cache_to_Delay   HitClass   0.0   0.0    0.0     0.0   0.4   0.0
8  CS_Cache_to_Delay  MissClass   0.0   0.0    0.0     0.0   0.6   0.0


In [8]:
# MVA
model.reset()
solver = np.append(solver, MVA(model, seed=1))
print('\nSOLVER: MVA')
AvgTable3 = solver[2].avg_node_table()


SOLVER: MVA
                Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0              Delay   JobClass   1.0   1.0    1.0  0.3333  0.3333  0.3333
1              Delay   HitClass   0.0   0.0    0.0  0.0000  0.3333  0.3333
2              Delay  MissClass   0.0   0.0    0.0  0.0000  0.3333  0.3333
4              Cache   HitClass   0.0   0.0    0.0  0.0000  0.0000  0.3333
5              Cache  MissClass   0.0   0.0    0.0  0.0000  0.0000  0.3333
6  CS_Cache_to_Delay   JobClass   0.0   0.0    0.0  0.0000  0.3333  1.0000
7  CS_Cache_to_Delay   HitClass   0.0   0.0    0.0  0.0000  0.3333  0.0000
8  CS_Cache_to_Delay  MissClass   0.0   0.0    0.0  0.0000  0.3333  0.0000
